# Using _egobox_ optimizer _Egor_

## Installation

In [1]:
%pip install egobox

Note: you may need to restart the kernel to use updated packages.


We import _egobox_ as _egx_ for short.

In [2]:
import numpy as np
import egobox as egx

You may setup the logging level to get optimization progress during the execution

In [3]:
# To display optimization information (none by default)
# import logging
# logging.basicConfig(level=logging.INFO)

## Example 1 : Continuous optimization

### Test functions

In [4]:
xspecs_xsinx = egx.to_specs([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [5]:
xspecs_g24 = egx.to_specs([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


### Continuous optimization with _Egor_

In [6]:
egor = egx.Egor(g24, xspecs_g24, 
                     n_doe=10, 
                     n_cstr=n_cstr_g24, 
                     cstr_tol=[1e-3, 1e-3],
                     infill_strategy=egx.InfillStrategy.WB2,
                     target=-5.5,
                     # outdir="./out",
                     # hot_start=True
                    )  # see help(egor) for options

# Specify regression and/or correlation models used to build the surrogates of objective and constraints
#egor = egx.Egor(g24, xlimits_g24, n_cstr=n_cstr_g24, n_doe=10,
#                      regr_spec=egx.RegressionSpec.LINEAR,
#                      corr_spec=egx.CorrelationSpec.MATERN32 | egx.CorrelationSpec.MATERN52)  

In [7]:
res = egor.minimize(n_iter=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-5.50837809e+00  3.84933482e-04  3.56699512e-04] at [2.329518   3.17886009]
Optimization history: 
Inputs = [[2.14941171 1.2022835 ]
 [2.89591634 3.04421182]
 [1.13129495 3.27983113]
 [2.50882074 2.47459348]
 [1.34917886 1.15978118]
 [1.99449618 0.14570364]
 [1.68323092 3.96900812]
 [0.26740239 1.98981025]
 [0.823056   0.52179224]
 [0.34794318 2.23644392]
 [2.3323044  3.09159185]
 [2.32962545 3.17922313]
 [2.329518   3.17886009]]
Outputs = [[-3.35169521e+00 -1.00398765e+00 -2.62111904e+00]
 [-5.94012816e+00 -1.24186360e+01  2.88844914e+00]
 [-4.41112609e+00 -6.51809729e-01  3.03904161e+00]
 [-4.98341421e+00 -2.78451535e+00  2.77667993e-01]
 [-2.50896004e+00 -2.38224715e+00 -1.69313518e-01]
 [-2.14019982e+00 -1.85453736e+00 -3.85405403e+00]
 [-5.65223904e+00  1.40041321e+00  7.31474755e-01]
 [-2.25721264e+00 -4.39484898e-01 -1.40405159e+01]
 [-1.34484824e+00 -3.35493157e+00 -7.17152437e-02]
 [-2.58438710e+00 -4.24396527e-01 -9.72535570e+00]
 [-5.42389625e+00 -1.09766687e

## Example 2 : Mixed-integer optimization

### Test function

In [8]:
xspecs_mixint_xsinx = [egx.XSpec(egx.XType.INT, [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    return y

### Mixed-integer optimization with _Egor_

In [9]:
egor = egx.Egor(mixint_xsinx, xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     target=-15.12,
                    )  # see help(egor) for options
res = egor.minimize(n_iter=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-15.12161154] at [19.]
Optimization history: 
Inputs = [[ 7.]
 [24.]
 [11.]
 [ 6.]
 [ 5.]
 [ 4.]
 [17.]
 [18.]
 [19.]]
Outputs = [[  3.14127616]
 [  4.91604976]
 [  5.1356682 ]
 [  1.78601478]
 [  0.68929352]
 [  0.07924194]
 [-12.35295142]
 [-14.43198471]
 [-15.12161154]]


## Example 3 : More mixed-integer optimization

In the following example we see we can have other special integer type cases, where a component of x can take one value out of a list of ordered values (ORD type) or being like an enum value (ENUM type). Those types differ by the processing related to the continuous relaxation made behind the scene:
* For INT type, resulting float is rounded to the closest int value,
* For ORD type, resulting float is cast to closest value among the given valid ones,
* For ENUM type, one hot encoding is performed to give the resulting value.  

### Test function

In [10]:
# Objective function which takes [FLOAT, ENUM1, ENUM2, ORD] as input
# Note that ENUM values are passed as indice value eg either 0, 1 or 2 for a 3-sized enum  
def mixobj(X):
    # float
    x1 = X[:, 0]
    #  ENUM 1
    c1 = X[:, 1]
    x2 = c1 == 0
    x3 = c1 == 1
    x4 = c1 == 2
    #  ENUM 2
    c2 = X[:, 2]
    x5 = c2 == 0
    x6 = c2 == 1
    # int
    i = X[:, 3]

    y = (x2 + 2 * x3 + 3 * x4) * x5 * x1 + (x2 + 2 * x3 + 3 * x4) * x6 * 0.95 * x1 + i
    return y.reshape(-1, 1)

### Mixed-integer optimization with _Egor_

In [11]:
xtypes = [
    egx.XSpec(egx.XType.FLOAT, [-5.0, 5.0]),
    egx.XSpec(egx.XType.ENUM, tags=["blue", "red", "green"]),
    egx.XSpec(egx.XType.ENUM, xlimits=[2]),
    egx.XSpec(egx.XType.ORD, [0, 2, 3]),
]
egor = egx.Egor(mixobj, xtypes, seed=42)
res = egor.minimize(n_iter=10)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-14.25] at [-5.  2.  1.  0.]
Optimization history: 
Inputs = [[-1.90197486  2.          1.          3.        ]
 [ 1.36933896  1.          0.          2.        ]
 [-0.10843099  1.          0.          0.        ]
 [-4.73477511  0.          0.          3.        ]
 [ 3.11266243  2.          1.          2.        ]
 [ 0.33069418  2.          1.          0.        ]
 [ 4.47594664  2.          1.          0.        ]
 [-3.26619512  0.          0.          2.        ]
 [-5.          2.          1.          2.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          1.          0.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]]
Outp

Note that `x_opt` result contains indices for corresponding optional tags list hence the second component should be read as 0="red", 1="green", 2="blue", while the third component was unamed 0 correspond to first enum value and 1 to the second one.

## Usage

In [12]:
help(egor)

Help on Egor in module builtins object:

class Egor(object)
 |  Egor(fun, xspecs, n_cstr=0, cstr_tol=None, n_start=20, n_doe=0, doe=None, regr_spec=Ellipsis, corr_spec=Ellipsis, infill_strategy=Ellipsis, q_points=1, par_infill_strategy=Ellipsis, infill_optimizer=Ellipsis, kpls_dim=None, n_clusters=1, target=Ellipsis, outdir=None, hot_start=False, seed=None)
 |  
 |  Optimizer constructor
 |  
 |  fun: array[n, nx]) -> array[n, ny]
 |       the function to be minimized
 |       fun(x) = [obj(x), cstr_1(x), ... cstr_k(x)] where
 |          obj is the objective function [n, nx] -> [n, 1]
 |          cstr_i is the ith constraint function [n, nx] -> [n, 1]
 |          an k the number of constraints (n_cstr)
 |          hence ny = 1 (obj) + k (cstrs)
 |       cstr functions are expected be negative (<=0) at the optimum.
 |  
 |   n_cstr (int):
 |       the number of constraint functions.
 |  
 |   cstr_tol (list(n_cstr,)):
 |       List of tolerances for constraints to be satisfied (cstr < t